### 0. Fuentes del profesor

https://loe.org/images/content/091023/Zombie%20Publication.pdf - Paper de "When Zombies Attack" cuyo modelo debemos reproducir


http://thetortoiseslens.blogspot.com/2010/03/agent-based-computational-model-of.html - Modelacion de Agentes en Punto 5.1

### 1. Reproduccion del Modelo
En el artículo _WHEN ZOMBIES ATTACK!: MATHEMATICAL MODELLING OF AN OUTBREAK OF ZOMBIE INFECTION_ se muestra el modelado básico del modelo **SIR** para una epidemia. Reproduzca los modelos ahí presentados. 


##### Fuentes Adicionales: 
Teoricamente: https://complejidad.iiec.unam.mx/cursotaller2020/casos/modeloSIR.php

Implementacion en Python: https://blog.devgenius.io/pandemic-modeling-with-python-the-sir-model-16b5f8348e85

https://python.quantecon.org/sir_model.html

https://towardsdatascience.com/infectious-disease-modelling-part-i-understanding-sir-28d60e29fdfc?gi=7d6da078a7f7

https://www.cracked.com/article_18683_7-scientific-reasons-zombie-outbreak-would-fail-quickly.html

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp

In [ ]:
N = 1000
beta = 0.5  #Infeccion por encuentro
alfa = 0.7 #Tasa de muerte zombie (i.e., que tan probable es matar a un zombie en un encuentro con el)
zeta = 0.2 #que tan probable es que alguien que muere de S reviva como zombie
#asumimos que los Z no pueden volver a revivir como Z
rho = 1.0 # Cuanto tarda un infectado en volverse zombie

#tasas de cuarentena
kappa = 0 # de infectados
sigma = 0 # de zombies

#tasa de intento de escape
gamma = 0

#tasa de curacion
c = 0

delta = 0 # la tasa de muerte natural/por causas no directamente atribuibles a encuentro con zombie
pi = 0 #Tasa de nacimiento

S0, I0, R0 = 999, 1, 0  # Condiciones Iniciales: 1 infectado, resto suceptible

In [3]:
#El caso 1 es el caso basico

class ODEs:
    
    def __init__(self, beta, alfa, zeta, rho=0, kappa=0, sigma=0, gamma=0, c=0, delta=0, pi=0):
        self.beta = beta
        self.alfa = alfa
        self.zeta = zeta
        self.rho = rho
        self.gamma = gamma
        
        self.kappa = kappa
        self.sigma = gamma
        
        self.c = c

        self.delta = delta
        self.pi = pi  
    
    def ode_caso_basico(self, S, Z, R):
        dSdt = self.pi - (self.beta * S * Z) - (self.delta * S)
        dZdt = (self.beta * S * Z) + (self.zeta * R) - (self.alfa*S*Z)
        dRdt = (self.delta * S) + (self.alfa * S * Z) - (self.zeta*R)
        return dSdt, dZdt, dRdt
    
    def ode_caso_latente(self, S, Z, I, R):
        dSdt = self.pi - (self.beta * S * Z) - (self.delta * S)
        dIdt = (self.beta * S * Z) - (self.rho*I) - (self.delta*I)
        dZdt = (self.rho*I) + (self.zeta * R) - (self.alfa*S*Z)
        dRdt = (self.delta * S) + (self.delta * I) + (self.alfa * S * Z) - (self.zeta*R)
        return dSdt, dIdt, dZdt, dRdt
    
    def ode_caso_cuarentena(self, S, Z, I, R):
        dSdt = self.pi - (self.beta * S * Z) - (self.delta * S)
        dIdt = (self.beta * S * Z) - (self.rho*I) - (self.delta*I) - (self.kappa * I)
        dZdt = (self.rho*I) + (self.zeta * R) - (self.alfa*S*Z) - (self.sigma*Z)
        dRdt = (self.delta * S) + (self.delta * I) + (self.alfa * S * Z) - (self.zeta*R) + (self.gamma * Q)
        
        dQdt = (self.kappa * I) + (self.sigma * Z)  - (self.gamma*Q)
        
        return dSdt, dIdt, dZdt, dRdt, dQdt

    def ode_caso_tratamiento(self, S, Z, I, R):
        dSdt = self.pi - (self.beta * S * Z) - (self.delta * S) + (self.c * Z)
        dIdt = (self.beta * S * Z) - (self.rho*I) - (self.delta*I)
        dZdt = (self.rho*I) + (self.zeta * R) - (self.alfa*S*Z) - (self.c * Z)
        dRdt = (self.delta * S) + (self.delta * I) + (self.alfa * S * Z) - (self.zeta*R)
        
        return dSdt, dIdt, dZdt, dRdt
    
    
    def ode_ataques(self, Z, k, n):
        zombies_erradicados = k*n*Z
        return zombies_erradicados
        
        
    def ode_colateral_de_ataques(self, S, I, collateral, n):
        muertes_en_combate = collateral * n * (S+I)
        return muertes_en_combate

In [45]:
N = 1000
beta = 1  #Infeccion por encuentro
alfa = 0.2 #Tasa de muerte zombie (i.e., que tan probable es matar a un zombie en un encuentro con el)
zeta = 0.8 #que tan probable es que alguien que muere de S reviva como zombie
#asumimos que los Z no pueden volver a revivir como Z
rho = 1.0 # Cuanto tarda un infectado en volverse zombie
c = 8 #tasa de curacion

escenario_optimista = ODEs(beta=beta, alfa=alfa, zeta=zeta, c=c)


S=999
I=10
Z=10
R=0

cambio_en_S = 0
cambio_en_I = 0
cambio_en_Z = 0
cambio_en_R = 0
while(S >= 0 and Z >= 0):
    S += cambio_en_S
    I += cambio_en_I
    Z += cambio_en_Z
    R += cambio_en_R
    print("Humanos: ", S)
    print("Infectados: ", I)
    print("Zombies: ", Z)
    cambio_en_S, cambio_en_I, cambio_en_Z, cambio_en_R = escenario_optimista.ode_caso_tratamiento(S, Z, I, R)
    
    
    
    
    

Humanos:  999
Infectados:  10
Zombies:  10
Humanos:  -8911
Infectados:  10000
Zombies:  -20050.0


In [19]:
print(cambio_en_S)

-89.9


### 2. ¿Cuáles son las diferencias entre una epidemia normal (influenza, gripe, peste) y una epidemia zombie?


### 3.Criticas
Se han hecho muchas críticas al artículo (e.g. _Zombie Apocalypse: An Epidemic Model_) indicando que hay errores y que las gráficas no corresponden ¿Es cierto? Si es afirmativa la respuesta, ¿Cuáles son esos errores?

## Le faltaron algunos casos: 

### El caso del tratamiento con inmunidad

### Las muertes colaterales cuando se ataca a los zombies

###  

### 4. Upgrades
De la presentación _Zombie Apocalypse: An Epidemic Model_ se presentan varios `upgrades`, desarróllalos y coméntalos ¿Por qué son `upgrades`? ¿Qué mejoras proponen?


### 5.1 **Agentes** 
Use la clase agente para modelar una epidemia Zombie como se describe en [Agent-Based Computational Model of Humanity’s Prospects for Post Zombie Outbreak Survival](http://thetortoiseslens.blogspot.mx/2010/03/agent-based-computational-model-of.html). Utiliza el diagrama del artículo _SimZombie: A Case-Study in Agent-Based Simulation Construction_ como guía.

### 5.2 **Agentes** 
Use la clase agente para modelar una epidemia Zombie como se describe en [Zombie Infection Simulator](http://asymptote.wordpress.com/2008/01/13/asymptotes-zombie-infection-simulator/) (Esta incluye humanos que pelean  y humanos en pánico)

7: Zombies have too many natural predators. Humans are only at the top of the food chain because of our intelligence. Zombies don't have that advantage, so any animal could easily kill a zombie (stray dogs, bears, ect.) Plus, insects.

6: They can't take the heat The natural decaying process, exasperated by the daily heat, would cause zombies to basically explode from gas build up. Also, zombies would dry up in dry heat like "slim jims".

5: They can't take the cold The water in the zombie's bodies would freeze if temps outside reached below the freezing mark. This would make the zombies immobile.

4: Biting is a terrible way to spread disease Infectious diseases which spread via biting have a terrible track record. Look at how rare rabies is. It's to obvious a way to spread disease, and the authorities would be able to quarantine, or execute, the infected quickly.

3: They can't heal from day to day damage Zombies aren't alive, so their body won't heal. Basic wounds would accumulate and destroy the body quite quickly.

2: The landscape is full of zombie proof barriers If the zombies can only travel on foot it'll be hard to see widespread, rapid spreading of the disease. Zombies would either walk off cliffs or similar large drops, or would be corralled into small enough areas by these landmarks to allow mass killings of the infected.

1: Weapons and the people who use them Lot of armed people in the States. 14 million people with a hunting license alone. Millions more that have weapons without licenses. The military is suited to take down very large numbers of zombies without getting close enough to get bitten. Zombies have to come in contact with a person to spread their disease, which makes them susceptible to being killed every time they want to claim a new victim.

### 6. Preguntas Extras

#### 6.1 ¿Cómo modelarías una infección de hombres lobos?
    - Tip: Necesitas llevar el paso del tiempo

#### 6.2 Implementa el agente hombre-lobo

#### 6.3 Discute el ejemplo de [z0mb1e](http://www.elliemary.com/agent-based-zombie-model.html), es una de las simulaciones más completas. ¿Qué hay de diferente a la tuya?

#### 6.4 Desarrolla el agente descrito en ese artículo.


#### 6.5 Desarrolla una simulación de dicho artículo.